## **Search Exercise 7**
# Sliding Blocks Puzzles

In this exercise we shall investigate a family of _**sliding blocks**_ puzzles, which can be regarded as a generalisation of puzzles such as **8-Puzzle** and **15-Puzzle**. We now consider puzzles in which the sliding elements are not only single squares but can also be any shape formed by joining several squares
(sometimes called _polyominoes_).

You can see a wide variety of examples of sliding block puzzles at:
* [Nick Baxter's Sliding Block Puzzle Page](https://www.johnrausch.com/SlidingBlockPuzzles/)

### Exercise Overview

This exercise will provide you with classes that can represent sliding blocks puzzles in a general way and include methods that interface the representation with Brandon's `bbSearch` module.

It will illustrate the use of `bbSearch` for this kind of problem with a few examples. It will then consider possible heuristics. You will see that event quite simple heuristics can give very significant performance improvements, compared to exhaustive un-informed search.

Having seen these examples you will be ready to do some investigation and experimentation on your own, including:

* Creating your own puzzles of different levels of difficulty.
* Testing different search algorithms and parameters.
* Devising heuristics to guide the search.
* Carrying out systemmatic comparisons between different search algorithms on a variety of different problems and using different heuristics.

### Setup

As usual we will be using `bbSearch.py`, which is loaded by the following cell. By now, if you have been doing the search exercises you should be familiar with how this software works. However, if you need some revision of how to run the search algorithm and the various options available, you should go back to **Search Exercise 2** which explains the basic usuage. And you may also want to go through some of the examples in exercises **3-6**.

In [ ]:
!echo Installing bbSearch module from web ...
!echo creating bbmodcache subfolder
!mkdir -p bbmodcache
!echo downloading bbSearch module
!curl http://bb-ai.net.s3.amazonaws.com/bb-python-modules/bbSearch.py > bbmodcache/bbSearch.py

from bbmodcache.bbSearch import SearchProblem, search

### State Representation:
We will represent a given state of a sliding blocks puzzle by an array stored as a list of lists --- a list of rows, with each row being a list of individual cells. (This representation has been chosen for clarity rather than efficiency.) Each cell will contain an integer, where `0` will represent an empty cell and positive integers will represent the colour of a _polyomino_ section contained in that cell.

So a state might be represented as follows:
```python
  [ [1,3,0,0,0,0,6],
    [1,3,4,4,0,0,0],
    [3,3,4,0,0,0,0],
    [0,4,4,2,0,0,0],
    [0,0,2,2,5,5,5] ]
```
You will see a colouful graphic representation of this puzzle state below, once we have defined a class of storing, manipulating and displaying puzzle states.


## The `BlockState` Class

Although the list-of-lists array format is a concise way to specify and store puzzle states, it will be convenient to define a class that can encapsulate the various methods that we will want to use to access, manipulate, and display the states of a Sliding Blocks puzzle.

Take a look at the following code cell that defines `BlockState`. You don't need to worry much about `figure` that creates a graphical display. However, it will be useful to look at: `possible_moves` and `next_state`, which will be used by the search algorithm, and it may also be interesting to look at `free_cell` and `free_block`, that are used in calculating `possible_moves`.

Note the global variable `COLORS`, which implicitly specifies a mapping from integers to colours that is used to specify board states and goals. Integer `n` corresponds to the `n`th colour name in the list.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from copy import deepcopy

plt.ioff()  ## Turn off immediate display of plots

COLORS = ["lightgray", "red", "blue", "green", "yellow",
          "orange", "purple", "pink", "brown"]

class BlockState:

      # Specify mapping from directions to grid coordinate offsets:
      neighbour_offset = {"left": (0,-1), "right": (0,+1), "down":(+1,0), "up":(-1,0)}

      def __init__( self, blockstate, colors=COLORS ):
        self.blockstate = blockstate
        self.nrows = len(blockstate)
        self.ncols = len(blockstate[0])
        self.blocknums = set().union(*[set(row) for row in blockstate])
        self.blocknums = self.blocknums - {0}
        self.blocknumlist = list(self.blocknums)
        self.colors = colors

      def __repr__(self):
        return( str( self.blockstate ))

      # Find the cells occupied by a given number
      def blockcells( self, blocknum ):
          blockcells = []
          for row in range(self.nrows):
            for col in range(self.ncols):
              if self.blockstate[row][col] == blocknum:
                blockcells.append((row,col))
          return blockcells

      # Test if a cell is free (unblocked) in a given direction
      # Free if not blocked by edge of grid or by a cell of different colour
      def free_cell( self, direction, cell ):
        row, col = cell
        offrow, offcol = BlockState.neighbour_offset[direction]
        neighrow, neighcol = (row + offrow, col + offcol)
        if not (0 <= neighrow < self.nrows): return False #at top or bottom
        if not (0 <= neighcol < self.ncols): return False #at left or right
        neighval = self.blockstate[neighrow][neighcol]
        # Neighboring cell must be empty or part of the same coloured block
        return  (neighval==0 or neighval==self.blockstate[row][col])

      def free_block( self, direction, blockn ):
          blockcells = self.blockcells(blockn)
          for cell in blockcells:
            if not self.free_cell(direction, cell):
              return False
          return True

      def possible_moves(self):
        moves = []
        for blocknum in self.blocknumlist:
          for direction in ["left", "right", "down", "up"]:
              if self.free_block(direction, blocknum):
                  moves.append((blocknum, direction))
        return moves

      def next_state(self, move):
          next_blockstate = deepcopy(self.blockstate)
          blockno, direction = move
          cells = self.blockcells(blockno)
          ## first clear all cells of the block (set to 0)
          for cell in cells:
            row, col = cell
            next_blockstate[row][col] = 0
          rowoff, coloff = BlockState.neighbour_offset[direction]
          ## now set all neighbour cells (in move direction) to be
          ## cells with the blocknumber
          for cell in cells:
            row, col = cell
            next_blockstate[row+rowoff][col+coloff] = blockno
          return BlockState(next_blockstate)

      def color_key(self):
          return {b:self.colors[b] for b in self.blocknumlist}

      def figure(self, scale=0.5):
          nrows = self.nrows
          ncols = self.ncols
          fig, ax = plt.subplots(figsize=(ncols*scale+0.1,nrows*scale+0.1))
          plt.close(fig)
          ax.set_axis_off() # Don't show border lines and coordinate values

          frame = patches.Rectangle((0,0),1,1, linewidth=5, edgecolor='k', facecolor='w')
          ax.add_patch(frame)

          for row in range(nrows):
            for col in range(ncols):
                greyrect = patches.Rectangle( (((col*0.9)/ncols)+0.05,
                                               (((nrows-row-1)*0.9)/nrows)+0.05 ),
                                            0.9/ncols, 0.9/nrows,
                                            linewidth=1, edgecolor="gray", facecolor="lightgray")
                ax.add_patch(greyrect)

          for row in range(nrows):
            for col in range(ncols):
                cellval = self.blockstate[row][col]
                if cellval > 0:
                  cellcol = COLORS[cellval]
                  rect = patches.Rectangle( (((col*0.9)/ncols)+0.05,
                                             (((nrows-row-1)*0.9)/nrows)+0.05 ),
                                            0.9/ncols, 0.9/nrows,
                                            linewidth=0, edgecolor=cellcol, facecolor=cellcol)
                  ax.add_patch(rect)
          return fig

      def display(self):
          display(self.figure())

## Creating and displaying a `BlockState` object

Using the `BlockState` class we can easily create and display a `BlockState` object corresponding to a given puzzle state. This is illustrated by the following code cell:

In [ ]:
state = [[1,3,0,0,0,0,6],
         [1,3,4,4,0,0,0],
         [3,3,4,0,0,0,0],
         [0,4,4,2,0,0,0],
         [0,0,2,2,5,5,5]]

bs = BlockState(state)
bs.display()

## The `SlidingBlocksPuzzle` class
Now we specify the `SlidingBlocksPuzzle` class, which defines all the methods required to be used by the `bbsearch.search` function. You will see that most of these refer to functions that have already been defined within the `BlockState` class, so the definitions here are relatively simple.

In [ ]:
from copy import deepcopy
class SlidingBlocksPuzzle( SearchProblem ):

    def __init__( self, initial_state, goal, colors=COLORS ):
        """
        The __init__ method must set the initial state for the search.
        Arguments could be added to __init__ and used to configure the
        initial state and/or other aspects of a problem instance.
        """
        self.initial_state = BlockState(initial_state, colors=colors)
        self.colors = colors
        self.goal = BlockState(goal)

    def info(self):
        print("Solve the following sliding blocks problem.")
        print("Get from this initial state:")
        self.initial_state.display()
        print("To a state incorporating the following block position(s):")
        self.goal.display()
        print("You need to slide the red block to cover the bottom right square.")

    def possible_actions(self, state):
        return state.possible_moves()

    def successor(self, state, action):
        """
        This takes a state and an action and returns the new state resulting
        from doing that action in that state. You can assume that the given
        action is in the list of 'possible_actions' for that state.
        """
        return state.next_state(action)

    def goal_test(self, state):
        """
        For the sliding blocks puzzles, the goal condition is reached when
        all block possitions specified in the given goal state are satisfied by
        the current state. But empty positions (ie 0s) in the goal are ignored,
        so can be occupied by blocks in the current sate.
        """
        for row in range(state.nrows):
          for col in range(state.ncols):
            goalnum = self.goal.blockstate[row][col]
            if goalnum==0:
              continue
            if goalnum != state.blockstate[row][col]:
              return False
        return True

    def bf_cost(self, path, state):
        """
        This is an optional method that you only need to define if you are using
        a cost based algorithm such as "uniform cost" or "A*". It should return
        the cost of reaching a given state via a given path.
        If this is not re-defined, it will is assumed that each action costs one unit
        of effort to perform, so it returns the length of the path.
        """
        return len(path)
    
    def aStar_cost(self, path, state):
        """
        This is an optional method that you only need to define if you are using
        a cost based algorithm such as "uniform cost" or "A*". It should return
        the cost of reaching a given state via a given path.
        If this is not re-defined, it will is assumed that each action costs one unit
        of effort to perform, so it returns the length of the path.
        """
        return len(path)

    def display_action(self, action):
        """
        You can set the way an action will be displayed in outputs.
        """
        print((self.colors[action[0]], action[1]))

    def display_state(self, state):
        """
        You can set the way a state will be displayed in outputs.
        """
        state.display()

    def display_state_path( self, actions ):
        """
        This defines output of a solution path when a list of actions
        is applied to the initial state. It assumes it is a valid path
        with all actions being possible in the preceeding state.
        You probably don't need to override this.
        """
        s = self.initial_state
        self.display_state(s)
        for a in actions:
            self.display_action(a)
            s = self.successor(s,a)
            self.display_state(s)

## A small and simple test case
We are now ready to try out the `SlidingBlocksPuzzle` class for a simple puzzle example. We specify the initial and goal states and use these to create an instance of `SlidingBlocksPuzzle`.

In [ ]:
small_initial = [[0,0,1,1],
                 [4,4,1,0],
                 [0,2,6,0],
                 [2,2,6,0]]

small_goal    = [[0,0,0,0],
                 [0,0,0,0],
                 [0,0,1,1],
                 [0,0,1,0]]

small_puzzle =  SlidingBlocksPuzzle( small_initial, small_goal )



## Ready to test
Let us now do a breadth first search to try to solve `small_puzzle`.

In [ ]:
small_search = search( small_puzzle, 'BF/FIFO', 10000000, loop_check=True, randomise=False, show_state_path=True, return_info=True)

## A bigger and harder test case

In [ ]:
big   = [[1,3,0,0,0,0,6],
         [1,3,4,4,0,0,0],
         [3,3,4,0,0,0,0],
         [0,4,4,2,0,0,0],
         [0,0,2,2,5,5,5]]

big_goal   = [[0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0],
              [0,0,0,0,0,0,1],
              [0,0,0,0,0,0,1]]

big_puzzle =  SlidingBlocksPuzzle( big, big_goal )

big_search = search( big_puzzle, 'BF/FIFO', 10000000,
                     loop_check=True, randomise=False, show_state_path=True, return_info=True)


## Adding a Heuristic

The following `red_right_heuristic` is that, given any block state, will return the number of columns that the red block (the block identified by colour `1` needs to travel to the right to reach the rightmost column.

Here, we are using a typical idea for constructing a heuristic. We are calculating a number that ignores a lot of the constraints that apply to an actual solution and thereby calculates a minimum bound on the number of moves required to reach the goal state. In this case the heuristic calculation ignores the issue that there may be other blocks than need to be moved and also ignores the fact that the red block may have to be moved up and down as well as left to right. But we are sure that at least one move will need to be used for each of the remaining columns that the red block needs to be moved to reach the right edge of the grid.  

In [ ]:
def red_right_heuristic(state):
    for row in state.blockstate:
      for i, col in enumerate(row):
          if col == 1:
            return 6-i



## See how the heuristic helps

Now we can search for a solution to `big_puzzle` using the simple, and seemingly quite crude `red_right_heuristic`. Let's see what happens:

In [ ]:


big_search_rr = search( big_puzzle, 'BF/FIFO', 10000000, heuristic=red_right_heuristic,
                     loop_check=True, randomise=False, show_state_path=True, return_info=True)



## Now an even harder case

Can this heuristic work for an even harder puzzle?

In [ ]:
big   = [[1,3,0,0,0,0,6],
         [1,3,4,4,0,0,0],
         [3,3,4,0,0,0,0],
         [0,4,4,2,0,0,0],
         [0,0,2,2,5,5,5]]

hard_goal   = [[6,2,0,0,0,0,0],
               [2,2,0,0,0,0,0],
               [0,0,0,0,0,0,0],
               [0,0,0,0,0,0,1],
               [0,0,0,0,0,0,1]]

hard_puzzle =  SlidingBlocksPuzzle( big, hard_goal )



hard_search = search( hard_puzzle, 'BF/FIFO', 10000000, heuristic=red_right_heuristic,
                     loop_check=True, randomise=False, show_state_path=True, return_info=True)


**What happens if you try to solve the `hard_puzzle` without the heuristic?**

## Improving the Heuristic
Even if we still don't have a good idea of what would be a really good heuristic we can still add some extra information to the heuristic function in some simple ways.
For example, the purple tile will need to be moved to its destination cell at  `(0,0)` and every row and collum it is away from that cell will take at least one move. So if we find its position we can add the Manhattan distance of its position to that destination.

Since red and purple will need to be moved separately, we can simply add the values of the heuristics regarding red and purple to get another admissible heuristic. (And this will dominate both heuristics based on the positions of either red or purple alone.)

The following code defines this heuristic for purple and also the combined `red_and_purple` heuristic.


In [ ]:
def purple_heuristic(state):
      for r, row in enumerate(state.blockstate):
        for c, col in enumerate(row):
          if col == 6:
            return r+c

def red_and_purple_heuristic(state):
  return (red_right_heuristic(state) + purple_heuristic(state))

clever_search = search( hard_puzzle, 'BF/FIFO', 10000000, heuristic = red_and_purple_heuristic,
                     loop_check=True, randomise=False, show_state_path=True, return_info=True)

## Collecting and Tabulating Results

As we saw in earlier exercercises, when researching search problems and algorithms, it can be informative to collect results of a series of tests in the form of a table. This is illustrated in the next code cell.

In [ ]:

TEST_RESULTS =[small_search, big_search, big_search_rr, hard_search, clever_search]

# Specify symbols for termination conditions:
short_tc = {"GOAL_STATE_FOUND"     : "Y",
            "NODE_LIMIT_EXCEEDED"  : "!",
            "SEARH-SPACE_EXHAUSTED": "x"}

print("\n                **TESTS SUMMARY**\n")

print("Test    #max   Result   #gen     #inQ    Time s")
for i, test in enumerate(TEST_RESULTS):
    max  = test['args']['max_nodes']
    tc  = test['result']['termination_condition']
    stc = short_tc[tc]

    ng  = test['search_stats']['nodes_generated']
    nq  = test['search_stats']['nodes_left_in_queue']
    time = round( test['search_stats']['time_taken'], 2 )
    print( f"{i:>3}: {max:>8}    {stc}  {ng:>8} {nq:>8}     {time} ")